![1764139471794](image/test/1764139471794.png)


In [17]:
# 导入库
import tensorflow as tf
import numpy as np
from torchsummary import summary

# 1.散点输入 定义输入数据
data = [
    [-0.5, 7.7],
    [1.8, 98.5],
    [0.9, 57.8],
    [0.4, 39.2],
    [-1.4, -15.7],
    [-1.4, -37.3],
    [-1.8, -49.1],
    [1.5, 75.6],
    [0.4, 34.0],
    [0.8, 62.3],
]
# 转化为数组
data = np.array(data)
# 提取x 和y
x_data = data[:, 0]
y_data = data[:, 1]

# 转化为tensorflow用的张量
x_train = tf.constant(x_data, dtype=tf.float32)
y_train = tf.constant(y_data, dtype=tf.float32)

# 可以将numpy数组或者tensorflow张量
dataset = tf.data.Dataset.from_tensor_slices(
    (
        x_train,
        y_train,
    ),
)
# 使用 tf.data.Dataset.from_tensor_slices 创建数据集
# from_tensor_slices() 返回一个 tf.data.Dataset 对象，这个对象是一个可迭代的数据集，包含了从 tensors 中切分出来的数据。每次从 dataset 中提取数据时，它会返回一个元素（例如：(input, label) 配对）。
dataset = tf.data.Dataset.from_tensor_slices(
    (
        # tensors（输入数据）：可以是一个或多个 NumPy 数组、TensorFlow 张量、Python 列表等，表示模型的输入数据和目标标签。
        # 如果传入多个数组或张量，它们的切片会一起组成数据集中的每个元素。
        # 在此例中，tensors 是一个包含 x_train 和 y_train 的元组。x_train 是输入数据（特征），y_train 是目标标签（输出）。这些数组会按元素拆分并一起组成数据集的元素。
        # 形状要求：
        # 如果 x_train 和 y_train 是 NumPy 数组或 TensorFlow 张量，它们的形状应该一致，确保每个输入数据样本都对应一个目标标签。
        x_train,  # 输入数据，通常是一个 NumPy 数组或 TensorFlow 张量，形状为 (num_samples, num_features)
        y_train,  # 目标数据（标签），通常是一个 NumPy 数组或 TensorFlow 张量，形状为 (num_samples,)
    )
)

# 额外的参数及解释：

# 1. batch_size：指定每个批次中包含多少个样本
#    - 在训练过程中，数据会被拆分成多个批次，每个批次包含 `batch_size` 个样本。
#    - batch_size 越大，内存占用越高，训练可能越稳定，但每次更新权重的频率越低。
#    - 通常 batch_size 设置为 32、64 或 128 等常见值。
dataset = dataset.batch(
    batch_size=32,
    drop_remainder=True,
)
# 解释：
# drop_remainder=True：如果数据集的样本数无法整除批次大小，是否丢弃最后一个批次中的不足样本数。默认为 False。

# 2. shuffle：指定是否在训练时对数据进行打乱
#    - shuffle=True 会在每个 epoch 结束后打乱数据，帮助模型避免对数据顺序产生依赖。
#    - shuffle(buffer_size=10000)：设置缓冲区大小，缓冲区大小越大，打乱效果越好。
dataset = dataset.shuffle(
    buffer_size=10000,
)

# 3. prefetch：预取数据，用于在训练过程中提前加载数据，避免数据加载成为瓶颈
#    - prefetch(buffer_size=tf.data.experimental.AUTOTUNE)：TensorFlow 会自动调整缓冲区大小，确保 CPU 和 GPU 能够并行工作，提高效率。
dataset = dataset.prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE,
)

# 4. num_parallel_calls：并行加载数据的数量
#    - num_parallel_calls=tf.data.experimental.AUTOTUNE：让 TensorFlow 自动决定并行处理数据的数量，通常用于加速数据处理。
#    - 如果不指定，数据加载将是串行的，速度可能较慢。
# dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# 5. drop_remainder：指定是否丢弃最后一个批次中不足 `batch_size` 个样本的数据
#    - drop_remainder=True：如果最后一个批次的样本数不足 `batch_size`，则丢弃这个批次。
#    - drop_remainder=False：保持最后一个批次，即使它的样本数不足 `batch_size`。
#    - 适用于训练和验证数据，保证每个批次的样本数一致。
# dataset = dataset.batch(32, drop_remainder=True)

# 现在可以迭代 dataset，查看每个批次的数据
for batch_x, batch_y in dataset:
    print(batch_x.shape, batch_y.shape)


# tf.keras.Sequential() 是一种模型定义方式，表示按顺序逐层堆叠网络层
# 这里我们创建一个简单的全连接层（Dense层）模型，它有一个输入层和一个输出层
#  (1,) 指的是（None，1）None代表参数的数量不知道有多少
model = tf.keras.Sequential(
    [
        # 定义一层全连接层（Dense layer）
        tf.keras.layers.Dense(
            1,  # 输出维度，表示该层输出一个标量（即只有一个神经元）
            input_shape=(
                1,
            ),  # 输入的形状为 (1,) 表示输入是一个一维的数值（标量），例如一个数值 [x]
        ),
    ]
)
# 定义损失函数和优化器

# 使用 SGD（随机梯度下降）优化器来训练模型
# 随机梯度下降（SGD）是一种常见的优化算法，它通过迭代地调整模型的权重来最小化损失函数
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01,  # 学习率：控制每次更新时权重调整的步幅大小。较高的学习率可能导致训练不稳定，较低的学习率可能导致训练过程过慢。
    # 如果不设置学习率，默认值为 0.01
)

# 通过 model.compile() 告诉模型使用哪个优化器和损失函数来进行训练
# 通过编译模型，模型会知道如何根据数据进行训练和权重更新
model.compile(
    optimizer=optimizer,  # 使用上面定义的优化器（SGD）
    loss="mean_squared_error",  # 损失函数：均方误差（MSE），适用于回归任务。它通过计算预测值和真实值之间的差距来评估模型的表现。
    # 损失函数的作用是衡量模型预测的输出和实际标签之间的误差。较小的损失值表示模型预测效果较好。
)
# 定义日志位置
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
#
# #4.开始迭代
epoches = 500
history = model.fit(
    x_train,  # 训练数据的输入部分，形状通常是 (num_samples, num_features)
    y_train,  # 训练数据的目标值（标签），形状通常是 (num_samples,)
    epochs=epoches,  # 训练的轮次，即模型在整个训练数据上训练的次数
    # batch_size=32,       # 每个批次的样本数量，通常选择 32、64 等常见值。如果为 None，则使用整个数据集训练（即每个 epoch 只使用一个批次）
    # validation_data=(x_val, y_val),  # 验证数据集，使用这个数据集来评估模型在每个 epoch 结束时的表现
    # validation_split=0.2,             # 在训练数据中划分出 20% 的数据作为验证集，剩余的用于训练
    verbose=0,  # 控制训练过程中的日志显示：0 = 不显示进度，1 = 显示进度条，2 = 每个 epoch 输出一行日志
    # callbacks=[early_stopping],  # 训练过程中使用的回调函数，例如 EarlyStopping 用于避免过拟合
    # shuffle=True            # 是否在每个 epoch 后打乱数据，通常设置为 True
    # initial_epoch=0         # 从哪个 epoch 开始训练，通常用于恢复训练
    callbacks=[tensorboard_callback],  # 生成tensorboard所用到的日志
)

# 1. 保存整个模型（包括结构、权重和训练配置） -> HDF5 格式
# 保存完整模型，包括网络结构、权重和训练配置（优化器、损失函数等）。
# 适用于跨平台保存和加载。文件扩展名为 .h5
model.save("./model/save_model.h5")
# 解释：
# model.save() 会将整个模型（结构 + 权重 + 配置）保存为一个文件，可以直接用 `model.load()` 重新加载模型。

# 2. 只保存模型的权重（不保存模型结构） -> HDF5 格式
# 这种方式只保存模型的权重，包括模型的参数（权重 w 和偏置 b）。
# 适用于你已经有模型结构代码，且只需要恢复训练过程中的权重。
model.save_weights("./model/model.weights.h5")
# 解释：
# model.save_weights() 只保存权重（没有保存模型的结构）。如果你想在其他地方加载权重，需要重新定义相同的模型结构。

# 3. 使用 TensorFlow 保存模型（推荐使用 .keras 扩展名）
# 适用于 TensorFlow 环境，可以使用 `.keras` 格式保存模型。此格式更适合在 TensorFlow 环境中进行部署（如 TensorFlow Serving）。
model.save("./model/save_model.keras")
# 解释：
# `.keras` 是 TensorFlow 推荐的保存格式，能够将整个模型（包括权重、结构和配置）保存到文件中。可以在不同的 TensorFlow 环境中加载。

# 4. 使用 TensorFlow 格式保存模型（使用 .h5 扩展名） -> HDF5 格式
# 如果你习惯使用 `.h5` 格式保存模型，这也是常见的选择，适合与旧的 TensorFlow 版本兼容。
model.save("./model/save_model.h5")
# 解释：
# `.h5` 格式仍然被广泛使用，特别是在 TensorFlow 2.x 版本之前。它保存了整个模型（包括结构和权重），适合跨平台部署。

# 5. 保存模型为 SavedModel 格式（TensorFlow 专有格式）
# SavedModel 格式适用于 TensorFlow 和 TensorFlow Serving，是 TensorFlow 官方推荐的模型保存格式。
# 适合生产环境部署，支持跨语言和跨平台的使用。
# save_format 参数已被弃用
# model.save("./saved_model", save_format="tf")
# 解释：
# `save_format="tf"` 指定使用 TensorFlow 原生格式（SavedModel）保存模型。SavedModel 格式会将模型保存到文件夹中，包括模型结构、权重和优化器配置等。

# 默认是GPU如果是cpu最好指定一下
# 如果安装了gpu版的pytorch 如果不指定可能会报错
# summary() 的常见输出包括：
# Layer (type)：显示每个网络层的类型（例如 Linear、Conv2d 等）。
# Output Shape：显示每个层的输出形状（例如 (10, 1)）。  如果是-1则表示该维度大小不固定
# ！！！！！这里一定要搞清楚Output Shape说明的是输出tesner的形状
# nn.Linear(1, 1)表示的是输入特征是1 输出特征也是1
# Param #：显示每个层的参数数量。
# Total params：显示模型的总参数数量。
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (20.00 B)

 Trainable params: 2 (8.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [8]:
# 模型的加载方式 # 方式1: HDF5文件 后缀名 .h5
# 加载参数、模型结构、训练的配置等等
# 和pytorch区别不用加载于原来的模型类给到，在tensorflow里面可以不给
import tensorflow as tf
import numpy as np

# 加载模型 方案1
loader_model = tf.keras.models.load_model("save_model.h5")
# 预测
input_data = np.array([[-0.5]])
pre = loader_model.predict(input_data)
# 从数组中提取出结果值
print(f"model result :{pre[0][0]:2.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
model result :5.411


In [ ]:
# 模型的加载方式 # 方式1: HDF5文件 后缀名 .h5
# 加载参数、模型结构、训练的配置等等
# 和pytorch区别不用加载于原来的模型类给到，在tensorflow里面可以不给
import tensorflow as tf
import numpy as np


def linear():
    input = tf.keras.layers.Input(shape=(1,), dtype=tf.float32)
    y = tf.keras.layers.Dense(1)(input)
    model = tf.keras.models.Model(inputs=input, outputs=y)
    return model


model = linear()
model.load_weights("model.weights.h5")
input_data = np.array([[-0.5]])
pre = model.predict(input_data)
# 从数组中提取出结果值
print(f"model result :{pre[0][0]:2.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
model result :5.411


In [ ]:
# 方案 1: 使用 tf.keras.Sequential 定义模型
# 这种方式简单明了，适合层按顺序堆叠的情况

import tensorflow as tf
import numpy as np

# 1. 输入数据
data = [
    [-0.5, 7.7],
    [1.8, 98.5],
    [0.9, 57.8],
    [0.4, 39.2],
    [-1.4, -15.7],
    [-1.4, -37.3],
    [-1.8, -49.1],
    [1.5, 75.6],
    [0.4, 34.0],
    [0.8, 62.3],
]
data = np.array(data)
x_data = data[:, 0]  # 提取第一列为输入特征
y_data = data[:, 1]  # 提取第二列为目标输出
# TensorFlow 张量
# 输出是什么类型？
x_train = tf.constant(
    x_data,
    dtype=tf.float32,
)
y_train = tf.constant(
    y_data,
    dtype=tf.float32,
)

# 2. 定义模型
# 使用 Sequential 模型，添加一个 Dense 层
# 使用 tf.keras.Sequential 来定义一个简单的顺序模型
# Sequential 模型适用于层按顺序堆叠的情况，结构简单
# 全连接层（Dense层）模型，它有一个输入层和一个输出层
#  (1,) 指的是（None，1）None代表参数的数量不知道有多少
model = tf.keras.Sequential(
    [
        # 定义一层全连接层（Dense layer）
        tf.keras.layers.Dense(
            1,  # 输出维度，表示该层输出一个标量（即只有一个神经元）
            input_shape=(
                1,
            ),  # 输入的形状为 (1,) 表示输入是一个一维的数值（标量），例如一个数值 [x]
        ),
        # tf.keras.layers.Dense(
        #     ** units,  # 输出空间的维度，通常表示神经元的数量
        #     ** activation=None,  # 激活函数，指定该层的激活函数（如 'relu', 'sigmoid', 'tanh' 等），默认为 None，表示线性激活函数
        #     ** use_bias=True,  # 是否使用偏置项，默认为 True，表示使用偏置项
        #     ** kernel_initializer="glorot_uniform",  # 权重矩阵的初始化方法，默认为 'glorot_uniform'
        #     ** bias_initializer="zeros",  # 偏置项的初始化方法，默认为 'zeros'
        #     ** kernel_regularizer=None,  # 权重矩阵的正则化方法，默认为 None，表示不进行正则化
        #     ** bias_regularizer=None,  # 偏置项的正则化方法，默认为 None，表示不进行正则化
        #     ** activity_regularizer=None,  # 激活值的正则化方法，默认为 None，表示不进行正则化
        #     ** kernel_constraint=None,  # 权重矩阵的约束条件，默认为 None，表示不对权重进行约束
        #     ** bias_constraint=None,  # 偏置项的约束条件，默认为 None，表示不对偏置进行约束
        #     ** kwargs  # 其他额外的关键字参数
        # ),
    ]
)

# 3. 编译模型
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01,
)
model.compile(
    optimizer=optimizer,
    loss="mean_squared_error",
)

# 4. 训练模型
epochs = 500
# dataset 可以直接代替 x 和 y 参数传递给 model.fit()
# 所以可以使用dataset先设置好数据结构如：shuffle  batch  prefetch
history = model.fit(
    x_train,
    y_train,
    epochs=epochs,
    # x=None,  # 输入数据  可以用dataset代替  可以是 NumPy 数组、TensorFlow 张量、tf.data.Dataset 或其他支持的数据类型。如果使用 tf.data.Dataset，则不需要显式传入 y，因为数据集已经包含标签。
    # y=None,  # 标签数据  可以用dataset代替  可以是 NumPy 数组、TensorFlow 张量、tf.data.Dataset 或其他支持的数据类型。如果使用 tf.data.Dataset，则不需要显式传入 y，因为数据集已经包含标签。
    # batch_size=None,  # 每个批次的样本数  每个批次的样本数。默认为 None，如果没有传入 batch_size，则数据会根据 x 和 y 的大小自动计算批次大小。
    # epochs=1,  # 训练周期数
    # verbose=1,  # 控制训练过程中日志输出的方式控制训练过程中的日志输出方式：0：不输出任何日志。|| 1：显示进度条，显示每个 epoch 的训练进度。|| 2：每个 epoch 输出一行日志，显示训练损失和其他指标
    # validation_data=None,  # 验证数据  验证数据，可以是元组 (x_val, y_val)，其中 x_val 和 y_val 是验证数据和标签。该数据用于评估模型在每个 epoch 结束后的性能，以便于监控过拟合
    # validation_split=0.0,  # 从训练数据中分割出部分数据作为验证集 从训练数据中分割一定比例的数据作为验证集进行验证。值的范围是 [0, 1]。如果设置为 0.2，表示 20% 的数据会被用作验证集，剩余 80% 用于训练。
    # shuffle=True,  # 是否打乱数据
    # callbacks=None,  # 用于训练过程中的回调函数（例如，早停 EarlyStopping、模型检查点 ModelCheckpoint 等）。这些回调函数在训练过程中执行自定义操作，如监控损失或精度，或者在验证性能不再提升时停止训练。
    # workers=1,  # 数据加载时的线程数用于数据加载的线程数。当数据集较大时，可以使用多线程来加速数据加载。默认为 1，表示只使用一个线程。增加该值可以加速数据加载，但可能会增加内存占用。
    # use_multiprocessing=False,  # 是否使用多线程 是否使用多进程来加载数据。使用多进程可能会加快数据加载，但有时会导致一些问题，尤其是在 Windows 系统中。默认为 False，表示不使用多进程。
    # **kwargs
)

# 5. 保存模型
model.save_weights("./model/model_def_1.weights.h5")  # 保存权重
model.save("./model/save_model_def_1.keras")  # 保存为 TensorFlow 格式

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 2927.6138
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2766.3123
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2614.1406

d:\CodeRelated\codeService\anaconda3\envs\machine-learning\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2614.1406
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2470.5776
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 2335.1316
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2207.3398
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2086.7656
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1972.9973
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1865.6472
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1764.3496
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1668.7600
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1578.5532
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1493.4236
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1413.0818
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1337.2561
Epoch 16/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1265.6896
Epoch 17/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - 

In [ ]:
# 方案 2: 使用 Input 层与 Sequential 模型结合
# 这种方式使用 tf.keras.Input 定义输入层，适合一些较复杂的输入形状

import tensorflow as tf
import numpy as np

# 1. 输入数据
data = [
    [-0.5, 7.7],
    [1.8, 98.5],
    [0.9, 57.8],
    [0.4, 39.2],
    [-1.4, -15.7],
    [-1.4, -37.3],
    [-1.8, -49.1],
    [1.5, 75.6],
    [0.4, 34.0],
    [0.8, 62.3],
]
data = np.array(data)
x_data = data[:, 0]
y_data = data[:, 1]
x_train = tf.constant(x_data, dtype=tf.float32)
y_train = tf.constant(y_data, dtype=tf.float32)

# 2. 定义模型
# 使用 Input 层和 Sequential 模型结合
input_layer = tf.keras.Input(
    shape=(1,)
)  # 定义输入层  Input 层 用来定义模型的输入形状。它通常作为模型的第一层，用来接收数据。
# Input 层本身 不做任何计算，它只是接收数据并将其传递给后续层。shape=(1,) 表示每个输入样本是一个具有 1 个特征的向量。
# 当数据传入时，TensorFlow 会根据这个定义自动处理数据的形状。
model = tf.keras.Sequential(
    [
        input_layer,
        tf.keras.layers.Dense(
            1
        ),  # 1 表示该层的输出单元数目（即输出神经元的数量），这里输出只有 1 个节点，通常用于回归问题（如预测一个连续值）。
    ]  # 将 Input 层作为第一个层  # Dense 层为输出
)

# 3. 编译模型
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="mean_squared_error")

# 4. 训练模型
epochs = 500
history = model.fit(x_train, y_train, epochs=epochs)

# 5. 保存模型
model.save_weights("./model/model_def_2.weights.h5")  # 保存权重
model.save("./model/save_model_def_2.keras")  # 保存为 TensorFlow 格式

In [ ]:
# 方案 3: 使用自定义 Model 类
# 适用于更灵活的模型定义，可以定义更复杂的前向传播逻辑

import tensorflow as tf
import numpy as np

# 1. 输入数据
data = [
    [-0.5, 7.7],
    [1.8, 98.5],
    [0.9, 57.8],
    [0.4, 39.2],
    [-1.4, -15.7],
    [-1.4, -37.3],
    [-1.8, -49.1],
    [1.5, 75.6],
    [0.4, 34.0],
    [0.8, 62.3],
]
data = np.array(data)
x_data = data[:, 0]
y_data = data[:, 1]
x_train = tf.constant(
    np.expand_dims(x_data, axis=1),
    dtype=tf.float32,
)  # 增加维度以适应模型
y_train = tf.constant(y_data, dtype=tf.float32)


# 2. 定义自定义模型类
class LinearModel(tf.keras.Model):
    def __init__(self):
        super(LinearModel, self).__init__()
        # 定义 Dense 层
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs):
        return self.dense(inputs)


# 3. 实例化模型
model = LinearModel()

# 4. 编译模型
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="mean_squared_error")

# 5. 训练模型
epochs = 500
history = model.fit(x_train, y_train, epochs=epochs)

# 6. 保存模型
model.save_weights("./model/model_def_3.weights.h5")  # 保存权重
model.save("./model/save_model_def_3.keras")  # 保存为 TensorFlow 格式

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 2868.2292
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2710.4380
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2561.5708
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2421.1169
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 2288.5981
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2163.5603
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 2045.5784
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1934.2496
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1829.1957
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1730.0588
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1636.5027
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1548.2097
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1464.8806
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1386.2336
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [ ]:
# 方案 4：使用函数API
import tensorflow as tf
import numpy as np

# 1. 输入数据
data = [
    [-0.5, 7.7],
    [1.8, 98.5],
    [0.9, 57.8],
    [0.4, 39.2],
    [-1.4, -15.7],
    [-1.4, -37.3],
    [-1.8, -49.1],
    [1.5, 75.6],
    [0.4, 34.0],
    [0.8, 62.3],
]
data = np.array(data)  # 将列表转换为 NumPy 数组
x_data = data[:, 0]  # 提取数据的第一列作为输入特征（x_data）
y_data = data[:, 1]  # 提取数据的第二列作为目标标签（y_data）

# 将 NumPy 数组转换为 TensorFlow 张量（dtype=tf.float32）
x_train = tf.constant(x_data, dtype=tf.float32)
y_train = tf.constant(y_data, dtype=tf.float32)

# 2. 定义模型使用函数式 API
input_layer = tf.keras.layers.Input(shape=(1,))  # 定义输入层，输入形状为 (1,)
output_layer = tf.keras.layers.Dense(
    1
)(  # Dense(1) 会对输入进行加权求和，并输出一个单一的结果（一个标量），通常用于回归任务。
    input_layer
)  # 定义输出层，1 个神经元的全连接层（Dense） 这一部分表示将 输入层（input_layer）作为输入传递给 Dense 层。
model = tf.keras.models.Model(  # 创建模型
    inputs=input_layer,  # 输入层
    outputs=output_layer,  # 输出层
)

# 3. 编译模型
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01
)  # 定义优化器，使用随机梯度下降（SGD），学习率为 0.01
model.compile(
    optimizer=optimizer, loss="mean_squared_error"
)  # 编译模型，使用均方误差作为损失函数

# 4. 训练模型
epochs = 500  # 设置训练轮数（epoch 数）
history = model.fit(
    x_train, y_train, epochs=epochs
)  # 使用训练数据进行训练，训练 500 轮

# 5. 保存模型
model.save_weights("./model/model_def_4.weights.h5")  # 保存模型的权重到文件
model.save("./model/save_model_def_4.keras")  # 保存整个模型（结构 + 权重 + 训练配置）

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 3123.2556
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2950.3955
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2787.3445
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2633.5417
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2488.4587
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2351.5964
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2222.4856
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2100.6831
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1985.7715
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1877.3578
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1775.0710
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1678.5621
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1587.5012
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1501.5779
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf